In [3]:
# example command to test the Mjj_Mjjj.py
# python Mjj_Mjjj.py -s=XToYHTo6B_MX-2400_MY-800 -i=/STORE/matej/H3_skims/2017/XToYHTo6B_MX-2400_MY-800/XToYHTo6B_MX-2400_MY-800.root -o=/users/ferencek/HHH/H3PO/Analysis
# python Mjj_Mjjj.py -s=JetHT2017B -i=/STORE/matej/H3_skims/2017/JetHT2017B/A8157861-8164-7F40-860B-F4482FAE26ED.root -o=/users/ferencek/HHH/H3PO/Analysis
# python Mjj_Mjjj.py -s=TTbarHadronic -i=/STORE/matej/H3_skims/2017/TTbarHadronic/1F279F2D-26A0-FF41-950C-A0F6E658E376.root -o=/users/ferencek/HHH/H3PO/Analysis

In [1]:
import numpy as np
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

In [2]:
#---------------------------------------------
# Selection cuts
#---------------------------------------------
higgs_mass = 125.
delta_r_cut = 0.8
min_jet_mass = 50.

# FatJet cuts
ptcut = 250.
etacut = 2.5
mass_cut = [100.,150.]
pNet_cut = 0.9105

# Resolved jet cuts
res_ptcut = 30.
res_etacut = 2.5
res_mass_cut = [90.,150.]
# loose cut = 0.0532, med_cut = 0.3040, tight_cut = 0.7476 , https://twiki.cern.ch/twiki/bin/view/CMS/BtagRecommendation106XUL17   
res_deepBcut = 0.0532
#---------------------------------------------


def closest(masses):
    delta = abs(higgs_mass - masses)
    min_delta = ak.min(delta, axis=1)
    is_closest = (delta == min_delta)
    return is_closest


def HbbvsQCD(fatjet):
    score = (fatjet.particleNetMD_Xbb/(fatjet.particleNetMD_Xbb+fatjet.particleNetMD_QCD))
    return score


def precut(fatjets):
    return (fatjets.pt>ptcut) & (np.absolute(fatjets.eta)<etacut)


def FailPassCategories(fatjets, jets=None):
    # sort the fat jets in the descending pNet HbbvsQCD score
    sorted_fatjets = fatjets[ak.argsort(-HbbvsQCD(fatjets),axis=-1)]

    # fail region: 0 fat jets passing the pNet cut
    # pass region: at least 1 fat jets passing the pNet cut
    fail_mask = (HbbvsQCD(sorted_fatjets[:,0])<pNet_cut)
    pass_mask = (HbbvsQCD(sorted_fatjets[:,0])>pNet_cut)
    if jets is not None:
        return fatjets[fail_mask], fatjets[pass_mask], jets[fail_mask], jets[pass_mask]
    else:
        return fatjets[fail_mask], fatjets[pass_mask]


def SR_mask(fatjets):
    return (fatjets.msoftdrop>=mass_cut[0]) & (fatjets.msoftdrop<=mass_cut[1])


def VR_boosted_mask(fatjets):
    return (((fatjets[:,0].msoftdrop<mass_cut[0]) | (fatjets.msoftdrop[:,0]>mass_cut[1])) & (fatjets[:,0].msoftdrop>min_jet_mass) 
    & (fatjets[:,1].msoftdrop>min_jet_mass) & ((fatjets[:,1].msoftdrop<mass_cut[0]) | (fatjets[:,1].msoftdrop>mass_cut[1])) 
    & (fatjets[:,2].msoftdrop>=mass_cut[0]) & (fatjets[:,2].msoftdrop<=mass_cut[1]))


def VR_semiboosted_mask(fatjets):
    return ((fatjets.msoftdrop<mass_cut[0]) | (fatjets.msoftdrop>mass_cut[1])) & (fatjets.msoftdrop>min_jet_mass)

In [4]:
events = NanoEventsFactory.from_root("/STORE/matej/H3_skims/2017/TTbarHadronic/DBADF3D8-7C75-F74B-99AF-7AF0D41083BE.root",schemaclass=NanoAODSchema,metadata={"dataset":""},entry_stop=None).events()

In [5]:
fatjets = events.FatJet

# fat jets preselection
good_fatjets = fatjets[precut(fatjets)]

# select events with at least 3 preselected fat jets
events_preselection =       events[ak.num(good_fatjets, axis=1)>2]
good_fatjets        = good_fatjets[ak.num(good_fatjets, axis=1)>2]

# apply the jet mass cut to the 3 leading (in pT) fat jets
good_fatjets = good_fatjets[:,0:3]
good_fatjets = good_fatjets[VR_semiboosted_mask(good_fatjets)]

# select events with exactly N_req good fat jets and select N_sel leading good fat jets
events_semiboosted_fatjets = events_preselection[ak.num(good_fatjets, axis=1)==3]
good_fatjets               =       (good_fatjets[ak.num(good_fatjets, axis=1)==3])[:,0:2]

# select jets from selected events with exactly N_req good fat jets
jets = events_semiboosted_fatjets.Jet

# apply preselection on the resolved jets
good_jets = jets[(jets.pt > res_ptcut) & (np.absolute(jets.eta) < res_etacut) & (jets.btagDeepB>res_deepBcut)]

# require that there are at least 2 good jets present in the event
good_fatjets = good_fatjets[ak.num(good_jets, axis=1)>1]
good_jets    =    good_jets[ak.num(good_jets, axis=1)>1]

# require jets to be away from fat jets
away_jets_mask = good_jets.nearest(good_fatjets).delta_r(good_jets)>delta_r_cut
good_jets = good_jets[away_jets_mask]

# require that there are at least 2 good away jets present in the event
good_fatjets = good_fatjets[ak.num(good_jets, axis=1)>1]
good_jets    =    good_jets[ak.num(good_jets, axis=1)>1]

# calculate mass of all possible jet pairs and select the pair which has the mass closest to the Higgs boson mass
dijets = ak.combinations(good_jets, 2, fields=['i0', 'i1'])
dijet_masses = (dijets['i0'] + dijets['i1']).mass
is_closest = closest(dijet_masses)
closest_dijets = dijets[is_closest]
# apply the jet mass cut to the closest dijets
good_dijets = closest_dijets[((closest_dijets['i0'] + closest_dijets['i1']).mass>=res_mass_cut[0]) & ((closest_dijets['i0'] + closest_dijets['i1']).mass<=res_mass_cut[1])]

# select events with at least 1 good dijet (by construction there can be at most 1 per event)
good_fatjets = good_fatjets[ak.num(good_dijets, axis=1)>0]
good_dijets  =  good_dijets[ak.num(good_dijets, axis=1)>0]

In [6]:
print(len(good_fatjets))

79


In [7]:
print(good_fatjets)

[[FatJet, FatJet], [FatJet, FatJet], ... [FatJet, FatJet], [FatJet, FatJet]]


In [15]:
# different ways to get the same info
print(good_fatjets[0][0].pt)
print(good_fatjets[0,0].pt)
print(good_fatjets[0,0]['pt'])
print(good_fatjets[0,0,'pt'])

661.5
661.5
661.5
661.5


In [18]:
# print the leading jet pT in selected events
for e in good_fatjets:
    print(e[0].pt)

661.5
732.5
549.5
481.5
567.5
653.5
723.0
321.25
1112.0
440.5
405.0
735.0
333.75
903.5
519.5
455.0
622.5
590.0
590.5
438.75
462.0
474.0
598.0
259.0
703.0
481.75
579.5
664.0
389.0
498.25
356.75
487.5
916.0
424.0
779.0
657.5
551.5
340.25
326.25
473.5
422.0
584.0
618.5
514.5
951.0
428.25
524.5
594.0
618.5
374.0
354.5
547.0
1048.0
716.5
455.75
864.0
868.5
619.0
359.75
548.5
638.5
402.75
909.0
310.75
332.0
374.5
523.5
718.0
517.0
662.5
630.5
541.0
331.5
483.5
771.0
505.75
710.5
675.0
538.0


In [20]:
# print events
for e in good_fatjets:
    print(e)

[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJet]
[FatJet, FatJe

In [22]:
# select the leading fat jet in each event
first_fatjets = good_fatjets[:,0:1]    
for e in first_fatjets:
    print(e)

[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]
[FatJet]


In [23]:
# WRONG syntax
#trijet_mass = (good_fatjets[:,0]+good_fatjets[:,1]+good_dijets['i0',0]+good_dijets['i1',0]).mass

# CORRECT syntax
trijet_mass = (good_fatjets[:,0]+good_fatjets[:,1]+good_dijets[:,0]['i0']+good_dijets[:,0]['i1']).mass